In [ ]:
from pathlib import Path
from scipy.io import loadmat
import sys
import os


dataset_path = Path('data') / 'data.mat'
if not dataset_path.exists():
    alt = Path.cwd().parent / 'data' / 'data.mat'
    if alt.exists():
        dataset_path = alt
    else:
        raise FileNotFoundError(f"data.mat not found under {Path.cwd()} or its parent")

notebook_path = os.getcwd() 
print (f"Current notebook path: {notebook_path}")
project_root = os.path.dirname(notebook_path)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print (f"Added {project_root} to sys.path")

mat_data = loadmat(dataset_path)
print(mat_data.keys())

In [ ]:
import torch
import matplotlib.pyplot as plt
from utils import trainer
from utils import utils
from Systems import DynamicSystem
import Filters
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator
import random

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
# Pro plnou CUDA reprodukovatelnost (volitelné, ale doporučené)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
# --------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

In [ ]:
mat_data = loadmat(dataset_path)

souradniceX_mapa = mat_data['souradniceX']
souradniceY_mapa = mat_data['souradniceY']
souradniceZ_mapa = mat_data['souradniceZ']
souradniceGNSS = mat_data['souradniceGNSS'] 

# --- KROK 1: Extrakce 1D os z 2D mřížek ---
# Získáme unikátní souřadnice pro osy X a Y.
# Pro osu X vezmeme první řádek z X matice.
# Pro osu Y vezmeme první sloupec z Y matice.
x_axis_unique = souradniceX_mapa[0, :]
y_axis_unique = souradniceY_mapa[:, 0]

print(f"Rozměry 1D osy X: {x_axis_unique.shape}")
print(f"Rozměry 1D osy Y: {y_axis_unique.shape}")
print(f"Rozměry 2D dat výšek Z: {souradniceZ_mapa.shape}")


# --- KROK 2: Vytvoření interpolačního objektu ---
# POZOR: Scipy očekává, že osy budou v pořadí (y, x), protože
# NumPy pole jsou indexována jako (řádek, sloupec), což odpovídá (y, x).
print("\nVytvářím interpolační funkci...")
terMap_interpolator = RegularGridInterpolator(
    (y_axis_unique, x_axis_unique),  # N-tice 1D os (nejprve Y, pak X)
    souradniceZ_mapa,
    bounds_error=False,  # NEVYHAZUJ CHYBU
    fill_value=np.nan
)
print("...interpolační funkce vytvořena.")

# --- KROK 3: Vytvoření finální, uživatelsky přívětivé funkce ---
def terMap(px, py):
    """
    Vypočítá nadmořskou výšku pro dané souřadnice (px, py)
    pomocí interpolace z mapy terénu.
    
    Funkce zvládne jak jednotlivé body, tak celé pole bodů.
    """
    # Spojíme vstupní body do formátu, kterému interpolátor rozumí:
    # pole o dvou sloupcích [y, x].
    points_to_query = np.column_stack((py, px))
    
    # Zavoláme interpolátor a vrátíme výsledek
    return terMap_interpolator(points_to_query)

# 3D model


In [ ]:
import torch
from math import pi
from Systems import DynamicSystemTAN

state_dim = 3
obs_dim = 1

F = torch.tensor([[1.0, 0.0, 0.0],
                       [0.0, 1.0, 0.0],
                       [0.0, 0.0, 0.96925]])

Q = torch.diag(torch.tensor([36.0, 36.0, 0.05]))
R = torch.tensor([[3.0]])
delta_pos = np.diff(souradniceGNSS[:2, :], axis=1)
u_pos = np.pad(delta_pos, ((0, 0), (0, 1)), 'constant')
u_bias = np.zeros((1, u_pos.shape[1]))
# Spojíme a převedeme na tensor
u_sequence_cpu = torch.from_numpy(np.vstack((u_pos, u_bias))).float()
u_sequence_cpu = u_sequence_cpu.T
# ### ZDE JE KLÍČOVÁ OPRAVA ###
# Přesuneme sekvenci vstupů na stejné zařízení jako model
u_sequence = u_sequence_cpu.to(device)
print(f"Tenzor u_sequence je nyní na zařízení: {u_sequence.device}")
print(f"Nový tvar u_sequence: {u_sequence.shape}")

# Počáteční podmínky
x_0 = torch.tensor([souradniceGNSS[0, 0], souradniceGNSS[1, 0], 0.0])
P_0 = torch.tensor([[25.0, 0.0, 0.0],
                    [0.0, 25.0, 0.0],
                    [0.0, 0.0, 10.0]])

# Nelineární funkce měření h(x)
def h_nl(x: torch.Tensor) -> torch.Tensor:
    # ... (vaše funkce h_nl je již v pořádku, protože používá x.device) ...
    px = x[:, 0]
    py = x[:, 1]
    b = x[:, 2]
    vyska_terenu_np = terMap(px.detach().cpu().numpy(), py.detach().cpu().numpy())
    vyska_terenu = torch.from_numpy(vyska_terenu_np).float().to(x.device)
    result = vyska_terenu + b
    result = result.unsqueeze(1) 
    # --- KONEC OPRAVY ---

    # Pojistka pro případ, že by terMap přesto vrátila NaN
    # Aplikujeme na již správně tvarovaný result
    nan_mask = torch.isnan(result)
    if nan_mask.any():
        # Lepší strategie pro NaN - např. nahradit průměrnou výškou mapy
        avg_height = torch.nanmean(torch.from_numpy(souradniceZ_mapa).float().to(x.device))
        print(f"Varování: NaN detekován v h_nl, nahrazuji hodnotou {avg_height:.2f}")
        result[nan_mask] = avg_height # Nahradíme NaN

    # Vracíme tenzor tvaru [B, 1]
    return result


x_axis_unique = souradniceX_mapa[0, :]
y_axis_unique = souradniceY_mapa[:, 0]


print("Vytvářím instanci DynamicSystem...")
system_model = DynamicSystemTAN(
    state_dim=state_dim,
    obs_dim=obs_dim,
    Q=Q.float(),
    R=R.float(),
    Ex0=x_0.float(),
    P0=P_0.float(),
    F=F.float(),
    h=h_nl,
    x_axis_unique=x_axis_unique,
    y_axis_unique=y_axis_unique,
    device=device  # Předáme definované zařízení
)

In [ ]:
TRAIN_SEQ_LEN = 100      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 100      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 500      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 300   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 10    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 1     # Pro robustní vyhodnocení

BATCH_SIZE = 8         # Dobrý kompromis

# Generování trénovacích dat
print(f"\nGeneruji {NUM_TRAIN_TRAJ} trénovacích trajektorií (délka {TRAIN_SEQ_LEN})...")
x_train, y_train = utils.generate_data_for_map_with_input(
    system=system_model,
    num_trajectories=NUM_TRAIN_TRAJ,
    u_sequence=u_sequence, # Předáme celou sekvenci u
    seq_len=TRAIN_SEQ_LEN  # Specifikujeme požadovanou délku
)
# Změna zde: Vybereme správnou délku u_train
u_train = u_sequence[:TRAIN_SEQ_LEN].unsqueeze(0).expand(NUM_TRAIN_TRAJ, -1, -1) 
print(f" Tvary dat: x={x_train.shape}, y={y_train.shape}, u={u_train.shape}")


# Generování validačních dat
print(f"\nGeneruji {NUM_VALID_TRAJ} validačních trajektorií (délka {VALID_SEQ_LEN})...")
x_val, y_val = utils.generate_data_for_map_with_input(
    system=system_model,
    num_trajectories=NUM_VALID_TRAJ,
    u_sequence=u_sequence,
    seq_len=VALID_SEQ_LEN 
)
# Změna zde: Vybereme správnou délku u_val
u_val = u_sequence[:VALID_SEQ_LEN].unsqueeze(0).expand(NUM_VALID_TRAJ, -1, -1)
print(f" Tvary dat: x={x_val.shape}, y={y_val.shape}, u={u_val.shape}")

# Generování testovacích dat
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií (délka {TEST_SEQ_LEN})...")
x_test, y_test = utils.generate_data_for_map_with_input(
    system=system_model,
    num_trajectories=NUM_TEST_TRAJ,
    u_sequence=u_sequence,
    seq_len=TEST_SEQ_LEN
)
# Změna zde: Vybereme správnou délku u_test
u_test = u_sequence[:TEST_SEQ_LEN].unsqueeze(0).expand(NUM_TEST_TRAJ, -1, -1)
print(f" Tvary dat: x={x_test.shape}, y={y_test.shape}, u={u_test.shape}")

# ==========================================================
# 5. VYTVOŘENÍ DATALOADERŮ
# ==========================================================
print("\n--- 5. Vytváření DataLoaderů ---")
# Vytvoření TensorDatasetů s x, y, a u
train_dataset = TensorDataset(x_train, y_train, u_train)
val_dataset = TensorDataset(x_val, y_val, u_val)
test_dataset = TensorDataset(x_test, y_test, u_test) # Pro pozdější vyhodnocení

# Vytvoření DataLoaderů
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False) # Batch size 1 pro testování

print("DataLoadery připraveny.")

In [ ]:

# # ==============================================================================
# # 1. KONFIGURACE TESTU
# # ==============================================================================
# TEST_SEQ_LEN = 400 # Změňte zpět na 100 nebo kolik potřebujete
# NUM_TEST_TRAJ = 1
# J_SAMPLES_TEST = 25

# # ==============================================================================
# # 2. PŘÍPRAVA DAT (OPRAVENO)
# # ==============================================================================
# print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")

# # Nyní předáme oříznutou sekvenci 'u', takže i 'x' a 'y' budou mít správnou délku.
# x_test, y_test = utils.generate_data(
#     system_model, 
#     num_trajectories=NUM_TEST_TRAJ,
#     seq_len=TEST_SEQ_LEN
# )

# test_dataset = TensorDataset(x_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
# print("Generování dat dokončeno.")

# print("y shape:", y_test.shape) # Mělo by být [1, 20, 1]
# print("x shape:", x_test.shape) # Mělo by být [1, 20, 3]


In [ ]:

# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# import numpy as np
# import os
# import random
# import csv
# from datetime import datetime
# import pandas as pd
# from copy import deepcopy
# from state_NN_models import StateKalmanNet

# # Nastavení seedu pro reprodukovatelnost tohoto běhu
# torch.manual_seed(42)
# np.random.seed(42)
# random.seed(42)
# state_knet = StateKalmanNet(system_model, device=device, hidden_size_multiplier=12).to(device)
# print(state_knet)
# trainer.train_state_KalmanNet(
#     model=state_knet, 
#     train_loader=train_loader, 
#     val_loader=val_loader, 
#     device=device, 
#     epochs=200, 
#     lr=1e-5,
#     early_stopping_patience=40,
#     clip_grad=1.0
# )

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
from state_NN_models import StateKalmanNet_v2_3D_tan

# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knet = StateKalmanNet_v2_3D_tan(system_model, device=device, hidden_size_multiplier=12, output_layer_multiplier=4, num_gru_layers=1).to(device)
print(state_knet)
EPOCHS = 200
LEARNING_RATE = 1e-4 # Zkus menší LR
CLIP_GRAD = 1.0
EARLY_STOPPING_PATIENCE = 40

# Volání upravené trénovací funkce
trained_knet_model = trainer.train_state_KalmanNet_with_input( # Použij nový název funkce
    model=state_knet,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=EPOCHS,
    lr=LEARNING_RATE,
    clip_grad=CLIP_GRAD,
    early_stopping_patience=EARLY_STOPPING_PATIENCE
)

state_knet = trained_knet_model

In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# from state_NN_models import StateBayesianKalmanNet
# import numpy as np
# import os
# import random
# import csv
# from datetime import datetime
# import pandas as pd
# from copy import deepcopy

# model_config = {
#     "hidden_size_multiplier": 10,
#     "output_layer_multiplier": 4,
#     "num_gru_layers": 1,
#     "init_min_dropout": 0.4,
#     "init_max_dropout": 0.6
# }

# train_config = {
#     "total_train_iter": 600,
#     "learning_rate": 1e-5,
#     "clip_grad": 1.00,
#     "J_samples": 10,
#     "validation_period": 20,
#     "logging_period": 20,
#     "warmup_iterations":0 # Trénuj prvních 400 iterací jen na MSE
# }

# # =================================================================================
# # KROK 3: SPUŠTĚNÍ JEDNOHO TRÉNINKOVÉHO BĚHU
# # =================================================================================

# print("="*80)
# print("Spouštím jeden plnohodnotný tréninkový běh...")
# print(f"Parametry modelu: {model_config}")
# print(f"Parametry tréninku: {train_config}")
# print("="*80)

# # Vytvoření modelu
# state_bkn_knet = StateBayesianKalmanNet(
#     system_model,
#     device=device,
#     **model_config
# ).to(device)

# # Spuštění tréninku
# # Používáme `run_training_session`, která vrací slovník s výsledky
# results = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(model=state_bkn_knet,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     device=device,
#     **train_config
# )

# # `run_training_session` automaticky načte nejlepší model zpět,
# # takže `state_bkn_knet` nyní obsahuje váhy nejlepšího modelu.
# trained_model = results['final_model']

# print("\n" + "="*80)
# print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
# print("="*80)
# print(f"Nejlepší model byl nalezen v iteraci: {results['best_iter']}")
# # --- Změněné klíče, aby odpovídaly return statementu ---
# print(f"Nejlepší dosažený validační ANEES: {results['best_val_anees']:.4f}")
# print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
# print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")
# print(f"  NLL na validační sadě:       {results['best_val_nll']:.4f}")
# print("="*80)

# print(trained_model)
# # Nyní můžeš s `trained_model` pokračovat, například ho vyhodnotit na testovací sadě.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader


# ==============================================================================
# 3. INICIALIZACE FILTRŮ
# ==============================================================================
ukf_ideal = Filters.UnscentedKalmanFilter(system_model)
pf_sir_ideal = Filters.ParticleFilter(system_model, num_particles=200000)

# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA (OPRAVENO)
# ==============================================================================
all_x_true_cpu = []
all_x_hat_ukf_ideal_cpu, all_P_hat_ukf_ideal_cpu = [], []
all_x_hat_pf_sir_ideal_cpu, all_P_hat_pf_sir_ideal_cpu = [], []
all_x_hat_classic_knet_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

state_knet.eval()


with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch,u_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        u_test_seq_gpu = u_test_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)


        # --- A. Bayesian KalmanNet (Trajectory-wise) ---
        # ensemble_trajectories = []
        # for j in range(J_SAMPLES_TEST):
        #     state_bkn_knet.reset(batch_size=1, initial_state=initial_state)
        #     current_x_hats = []
        #     for t in range(1, TEST_SEQ_LEN):
        #         x_filtered_t, _ = state_bkn_knet.step(y_test_seq_gpu[t, :].unsqueeze(0))
        #         current_x_hats.append(x_filtered_t)
        #     ensemble_trajectories.append(torch.cat(current_x_hats, dim=0))
        # ensemble = torch.stack(ensemble_trajectories, dim=0)
        # predictions_bkn = ensemble.mean(dim=0)
        # diff = ensemble - predictions_bkn.unsqueeze(0)
        # covariances_bkn = (diff.unsqueeze(-1) @ diff.unsqueeze(-2)).mean(dim=0)
        # full_x_hat_bkn = torch.cat([initial_state, predictions_bkn], dim=0)
        # full_P_hat_bkn = torch.cat([system_model.P0.unsqueeze(0), covariances_bkn], dim=0)


        # --- B. Klasický StateKalmanNet (pouze MSE) ---
        state_knet.reset(batch_size=1, initial_state=initial_state)
        classic_knet_preds = []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t = state_knet.step(y_test_seq_gpu[t, :].unsqueeze(0), u_t=u_test_seq_gpu[t, :].unsqueeze(0))
            classic_knet_preds.append(x_filtered_t)
        full_x_hat_classic_knet = torch.cat([initial_state, torch.cat(classic_knet_preds, dim=0)], dim=0)
        

        ukf_i_res = ukf_ideal.process_sequence(
        y_seq=y_test_seq_gpu,
        u_sequence=u_test_seq_gpu, # <-- DŮLEŽITÉ PŘEDÁNÍ VSTUPU
        Ex0=system_model.Ex0,
        P0=system_model.P0
        )
        
        full_x_hat_ukf_i = ukf_i_res['x_filtered']
        full_P_hat_ukf_i = ukf_i_res['P_filtered']

        pf_sir_i_res = pf_sir_ideal.process_sequence(
        y_seq=y_test_seq_gpu,
        u_sequence=u_test_seq_gpu, # <-- DŮLEŽITÉ PŘEDÁNÍ VSTUPU
        Ex0=system_model.Ex0, 
        P0=system_model.P0
        )
        
        full_x_hat_pf_sir_i = pf_sir_i_res['x_filtered']
        full_P_hat_pf_sir_i = pf_sir_i_res['P_filtered']
        full_particles_history_pf_sir_i = pf_sir_i_res['particles_history']
        print(f"PF-SIR (ideální model) dokončen pro trajektorii {i + 1}/{NUM_TEST_TRAJ}.")

        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        # all_x_hat_bkn_cpu.append(full_x_hat_bkn.cpu()); all_P_hat_bkn_cpu.append(full_P_hat_bkn.cpu())
        all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet.cpu())
        all_x_hat_ukf_ideal_cpu.append(full_x_hat_ukf_i.cpu()); all_P_hat_ukf_ideal_cpu.append(full_P_hat_ukf_i.cpu())
        all_x_hat_pf_sir_ideal_cpu.append(full_x_hat_pf_sir_i.cpu()); all_P_hat_pf_sir_ideal_cpu.append(full_P_hat_pf_sir_i.cpu())
        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
# Seznamy pro sběr metrik
mse_bkn, anees_bkn = [], []; mse_ukf_ideal, anees_ukf_ideal = [], []; mse_classic_knet = []

mse_pf_sir_ideal, anees_pf_sir_ideal = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        def get_metrics(x_hat, P_hat):
            # Délka pravdivé trajektorie
            true_len = x_true.shape[0]
            
            # Ořízneme odhady, aby měly stejnou délku jako pravdivá data.
            # Bereme prvních `true_len` prvků (tj. pro t=0 až t=299).
            x_hat_aligned = x_hat[:true_len]
            P_hat_aligned = P_hat[:true_len]
            
            # Nyní mají všechny tenzory shodnou délku
            mse = F.mse_loss(x_hat_aligned, x_true).item()
            anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_aligned.unsqueeze(0), P_hat_aligned.unsqueeze(0))
            return mse, anees

        # Výpočty pro všechny modely
        # mse, anees = get_metrics(all_x_hat_bkn_cpu[i], all_P_hat_bkn_cpu[i]); mse_bkn.append(mse); anees_bkn.append(anees)
        mse = F.mse_loss(all_x_hat_classic_knet_cpu[i][1:], x_true[1:]).item(); mse_classic_knet.append(mse)
        mse, anees = get_metrics(all_x_hat_ukf_ideal_cpu[i], all_P_hat_ukf_ideal_cpu[i]); mse_ukf_ideal.append(mse); anees_ukf_ideal.append(anees)
        mse, anees = get_metrics(all_x_hat_pf_sir_ideal_cpu[i], all_P_hat_pf_sir_ideal_cpu[i]); mse_pf_sir_ideal.append(mse); anees_pf_sir_ideal.append(anees)
      
def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 80)
print("-" * 80)
print(f"{'--- Model-Based Filters ---':<35} | {'':<20} | {'':<20}")
# print(f"{'Bayesian KNet (BKN)':<35} | {avg(mse_bkn):<20.4f} | {avg(anees_bkn):<20.4f}")
print(f"{'KNet (pouze MSE)':<35} | {avg(mse_classic_knet):<20.4f} | {'N/A':<20}")
print("-" * 80)
print(f"{'--- Benchmarks ---':<35} | {'':<20} | {'':<20}")
print(f"{'UKF (Ideální model)':<35} | {avg(mse_ukf_ideal):<20.4f} | {avg(anees_ukf_ideal):<20.4f}")
print(f"{'PF-SIR (Ideální model)':<35} | {avg(mse_pf_sir_ideal):<20.4f} | {avg(anees_pf_sir_ideal):<20.4f}")
print("="*80)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ==============================================================================
# 1. PŘÍPRAVA DAT (stejná jako předtím)
# ==============================================================================
# Předpokládáme, že tyto proměnné již existují z vašeho vyhodnocení
# all_x_true_cpu, full_x_hat_classic_knet, full_x_hat_bkn

x_true_plot = all_x_true_cpu[0].numpy()
x_knet_plot = full_x_hat_classic_knet.cpu().numpy()
x_pf_sir_plot = all_x_hat_pf_sir_ideal_cpu[0].numpy()
# x_knet_plot = full_x_hat_bkn.cpu().numpy()

# Vytvoříme časovou osu pro osu X grafů
num_steps = x_true_plot.shape[0]
time_axis = np.arange(num_steps)

# Seznam názvů pro jednotlivé složky stavu (pro popisky os)
state_labels = [
    'Pozice X [m]',
    'Pozice Y [m]',
    'Bias barometru [m]'
]

# ==============================================================================
# 2. VYTVOŘENÍ 4x1 SUBPLOT GRAFU
# ==============================================================================
# fig, axes = plt.subplots(počet_řádků, počet_sloupců, ... )
fig, axes = plt.subplots(3, 1, figsize=(12, 18), sharex=True)
fig.suptitle('Detailní porovnání odhadů stavu v čase', fontsize=16)

# Smyčka přes všechny 4 složky stavu
for i in range(3):
    ax = axes[i] # Vybereme aktuální podgraf (osu)

    # A. Vykreslení skutečné (referenční) hodnoty
    ax.plot(time_axis, x_true_plot[:, i], 'r-', linewidth=2.5, label='Referenční hodnota')

    # B. Vykreslení odhadu z klasického KNetu
    ax.plot(time_axis, x_knet_plot[:, i], 'g--', linewidth=2, label='Odhad KNet')
    
    ax.plot(time_axis, x_pf_sir_plot[:, i], 'b-.', linewidth=2, label='Odhad PF-SIR')

    # Nastavení popisků a mřížky pro aktuální podgraf
    ax.set_ylabel(state_labels[i])
    ax.grid(True)
    ax.legend()

# Nastavení popisku pro sdílenou osu X (pouze u spodního grafu)
axes[-1].set_xlabel('Časový krok [s]')

# Zlepšíme rozložení, aby se popisky nepřekrývaly
plt.tight_layout(rect=[0, 0.03, 1, 0.96]) # Udělá místo pro hlavní název

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

# --- Předpokládáme, že tyto proměnné již existují z vašeho vyhodnocení ---
# all_x_true_cpu: Seznam s pravdivou trajektorií
# full_x_hat_classic_knet: Tenzor s odhady z klasického KNetu
# full_x_hat_bkn: Tenzor s odhady z Bayesian KNetu (předpoklad)
# souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa: Data mapy
# terMap: Vaše interpolační funkce

# --- Krok 1: Příprava dat ---
x_true_plot = all_x_true_cpu[0].numpy()
x_knet_plot = full_x_hat_classic_knet.cpu().numpy()

print(f"Tvar skutečné trajektorie: {x_true_plot.shape}")
print(f"Tvar odhadnuté trajektorie (KNet): {x_knet_plot.shape}")

# --- Krok 2: Vytvoření 3D grafu ---
fig = plt.figure(figsize=(14, 12))
ax = fig.add_subplot(111, projection='3d')

# Vykreslení povrchu terénu (volitelné)
ax.plot_surface(souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa, 
                  rstride=100, cstride=100, cmap='terrain', alpha=0.3)

# --- Krok 3: Vykreslení trajektorií ---

# A. Skutečná (referenční) trajektorie
px_true = x_true_plot[:, 0]
py_true = x_true_plot[:, 1]
pz_true = terMap(px_true, py_true)
ax.plot(px_true, py_true, pz_true, 'r-', linewidth=3, label='Referenční trajektorie')

# B. Odhadnutá trajektorie z KalmanNetu
px_knet = x_knet_plot[:, 0]
py_knet = x_knet_plot[:, 1]
pz_knet = terMap(px_knet, py_knet)
ax.plot(px_knet, py_knet, pz_knet, 'g--', linewidth=3, label='Odhad KNet')


# --- Krok 4: Finalizace grafu ---
ax.plot([px_true[0]], [py_true[0]], [pz_true[0]], 
        'o', color='black', markersize=10, label='Start')

ax.set_xlabel('Souřadnice X [m]')
ax.set_ylabel('Souřadnice Y [m]')
ax.set_zlabel('Nadmořská výška Z [m]')

# Upravíme název, aby zahrnoval všechny modely
ax.set_title('Porovnání referenční trajektorie a odhadů KNet/BKN') 
ax.legend()
ax.grid(True)

ax.view_init(elev=30., azim=-60)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

# --- Předpokládáme, že tyto proměnné již existují z vašeho vyhodnocení ---
# all_x_true_cpu: Seznam s pravdivou trajektorií
# full_x_hat_classic_knet: Tenzor s odhady z klasického KNetu
# full_x_hat_bkn: Tenzor s odhady z Bayesian KNetu (předpoklad)
# souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa: Data mapy
# terMap: Vaše interpolační funkce

# --- Krok 1: Příprava dat ---
x_true_plot = all_x_true_cpu[0].numpy()

print(f"Tvar skutečné trajektorie: {x_true_plot.shape}")


# --- Krok 2: Vytvoření 3D grafu ---
fig = plt.figure(figsize=(14, 12))
ax = fig.add_subplot(111, projection='3d')

# Vykreslení povrchu terénu (volitelné)
ax.plot_surface(souradniceX_mapa, souradniceY_mapa, souradniceZ_mapa, 
                  rstride=100, cstride=100, cmap='terrain', alpha=0.3)

# --- Krok 3: Vykreslení trajektorií ---

# A. Skutečná (referenční) trajektorie
px_true = x_true_plot[:, 0]
py_true = x_true_plot[:, 1]
pz_true = terMap(px_true, py_true)
ax.plot(px_true, py_true, pz_true, 'r-', linewidth=3, label='Referenční trajektorie')


# --- Krok 4: Finalizace grafu ---
ax.plot([px_true[0]], [py_true[0]], [pz_true[0]], 
        'o', color='black', markersize=10, label='Start')

ax.set_xlabel('Souřadnice X [m]')
ax.set_ylabel('Souřadnice Y [m]')
ax.set_zlabel('Nadmořská výška Z [m]')

# Upravíme název, aby zahrnoval všechny modely
ax.set_title('Porovnání referenční trajektorie a odhadů KNet/BKN') 
ax.legend()
ax.grid(True)

ax.view_init(elev=30., azim=-60)

plt.show()